In [1]:
#import packages

In [2]:
from cgi import test
from optparse import Values
from sqlite3 import Row
from time import time
import tkinter as tk
from turtle import width
import pyvisa
import time 
# import keyboard
import time
import cv2
from PIL import Image, ImageTk
from webbrowser import get
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import threading as th
import minimalmodbus
from skopt import Optimizer
from skopt.plots import plot_objective
from skopt.plots import plot_evaluations
# from simple_pid import PID
import os

C:\Users\Osuji\AppData\Local\Temp\ipykernel_10892\4243695268.py:1: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  from cgi import test


In [3]:
#connecting to devices 

In [4]:
rm = pyvisa.ResourceManager()

In [5]:
print(rm.list_resources()) 

('ASRL1::INSTR', 'ASRL6::INSTR', 'ASRL7::INSTR', 'ASRL13::INSTR', 'ASRL18::INSTR')


In [6]:
#ASRL7 = Linear Stage
com_pxpy = rm.open_resource('ASRL7::INSTR')
com_pxpy.baud_rate = 921600 #fixed

In [7]:
#syringe pump
com_syr = rm.open_resource('ASRL4::INSTR')
# com_syr.baud_rate = 115200

VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

In [12]:
#temperature control
com_hot = minimalmodbus.Instrument('com17', 1, minimalmodbus.MODE_RTU)
com_hot.serial.baudrate = 115200

In [8]:
#Turns on Motion Controller
com_pxpy.write('1MO')
com_pxpy.write('2MO')

#moving to different positions from live positions ##PR moves from relative position
#com_pxpy.write('1PA'+position_x)
#com_pxpy.write('2PA'+position_y)

5

In [ ]:
#we are choosing to make flush our 0 for syringe + camera ref 

In [ ]:
#Identify current position and print
live_position_x_t = com_pxpy.query_ascii_values('1TP?')
live_position_x = ''.join(str(element) for element in live_position_x_t)
live_position_y_t = com_pxpy.query_ascii_values('2TP?')
live_position_y = ''.join(str(element) for element in live_position_y_t)
print('(x,y):')
print(live_position_x, live_position_y)

In [9]:
def move_to(position_x, position_y):
    position_x = str(position_x)
    position_y = str(position_y)
    com_pxpy.write('1PA'+position_x)
    time.sleep(0.1)              
    com_pxpy.write('2PA'+position_y)
    time.sleep(0.1)

In [10]:
#align flush to camera
#Identify Camera Position
#I guess we should move to the camera and then run this command ??
move_to(-40,0) #locate camera 

In [11]:
#Temp testing


In [ ]:
camera_position_x = com_pxpy.query_ascii_values('1TP?')
camera_position_x = ''.join(str(element) for element in camera_position_x)
camera_position_y = com_pxpy.query_ascii_values('2TP?')
camera_position_y = ''.join(str(element) for element in camera_position_y)
print(camera_position_x,camera_position_y)

In [ ]:
#align flush to syringe
#Identify Syringe Position
move_to(0,14) # locate syringe position

In [ ]:
syringe_position_x = com_pxpy.query_ascii_values('1TP?')
syringe_position_x = ''.join(str(element) for element in syringe_position_x)
syringe_position_y = com_pxpy.query_ascii_values('2TP?')
syringe_position_y = ''.join(str(element) for element in syringe_position_y)
print(syringe_position_x,syringe_position_y)

In [ ]:
def capture_and_save_image(image_name):
    save_path = "directory_name_here"  # Specify the directory
    
    cap = cv2.VideoCapture(0)
      # Capture frame from the camera
    ret, frame = cap.read()
     # Save the captured frame with the specified name
    image_path = f"{save_path}/{image_name}.jpg"
    cv2.imwrite(image_path, frame)
    # Release the camera
    cap.release()


In [ ]:
#Identify Flush Position
#move to the camera and then run this command
#move_to(,) # locate flush

In [ ]:
#flush_position_x = com_pxpy.query_ascii_values('1TP?')
#flush_position_x = ''.join(str(element) for element in flush_position_x)
#flush_position_y = com_pxpy.query_ascii_values('2TP?')
#flush_position_y = ''.join(str(element) for element in flush_position_y)

In [ ]:
#Initialize Syringe Parameters
sy_dia = 10.3 #mm #Hamilton 81520

res_vol = 4 #mL #same for 1,2 

In [ ]:
#Defining an operation that executes the pumping procedure
def single_pump(sy_dia, res_vol, ID, rate, dispense_vol):
    #sy_dia = 12.03

    #ID = 0, 1, 2 (pump number)
    #res_vol = reservoir volume in ml (usually ~5), usually integer
    #sy_dia = diameter of syrine, current config has sy_dia = 12.03
    #rate = pump rate in ul/min (format: xx.x) - float, 1 decimal place
    #sample_vol = volume to be pumped at defined rate (format: xxx.x) - float, 1 decimal place

    com_syr.write(str(ID) + 'svolume ' + str(res_vol) + ' ml')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'diameter ' + str(float(sy_dia)))
    time.sleep(0.1)

    com_syr.write(str(ID) + 'irate ' + str(float(rate)) + ' ul/min')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'tvolume ' + str(float(dispense_vol)) + ' ul')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'irun')

In [ ]:
#Flush out the piping for a fresh dropcast
def flush(flush_volume_0, flush_volume_1, flush_time): #flushes syringe 1 & 2
    #minki made flush_time = 3*flush_volume (around 20s)
    
    
    rate_flush_0 = 60 * (flush_volume_0 / flush_time)
    rate_flush_1 = 60 * (flush_volume_1 / flush_time)
    
    #flush sample
    single_pump(sy_dia, res_vol, 0, rate_flush_0, flush_volume_0)
    time.sleep(0.1)

    single_pump(sy_dia, res_vol, 1, rate_flush_1, flush_volume_1)
    time.sleep(0.1)

In [ ]:
L_pipes = 400 #mm
V_flush_mL = 0.495 * L_pipes * 0.001 #mL #1/16 system
V_flush_uL = V_flush_mL * 1000

V_flush_initial = V_flush_uL + 10  

V_mixing_chamber = 0.57 #uL
V_0_flush = V_1_f = 3 * V_mixing_chamber #uL
V_1_flush = V_2_f = 3 * V_mixing_chamber #uL

flush_time = f_t = 20 #s
wait_flush = 240 #s

In [ ]:
#Initial flsh through pipes
move_to(syringe_position_x,syringe_position_y)
time.sleep(0.2)
flush(V_flush_initial, V_flush_initial, 30) #is time good? #initially 30
time.sleep(0.2)

In [ ]:
V_flush_initial

In [ ]:
#flow and droplet parameters 

dispense_time = t_d = 60 #s

total_dispense_vol = V_d = 20 #uL

Q_d = V_d/t_d #mL/min

evaporation_time = e_t = 10 #=  18000 #s
sample_distance = 20 #mm #center-center distance of droplets
flush_d = 30 #flush to sample 0-0 distance

#(figure the positioning out)^^

In [ ]:
#AE parameters 
file_path = "image"
#max_iteration = 10
#p = [0.5]

#p = [0,0.2,0.4,0.6,0.8,1] # 6 percentages

p = [.4,.5,.6,.7]
#T = []

In [ ]:
#Dropcasting Loop
for i in range(4):
    print(f'iteration {i+1}')
    time.sleep(0.2)
    
    #move to flush position
    print('flushing')
    move_to(syringe_position_x,syringe_position_y)
    time.sleep(0.2)
    flush(V_0_flush, V_1_flush, flush_time)
    time.sleep(0.2)
    time.sleep(10)
    
    #find location of each droplet in iteration
    n = 2 #number of droplets per row
    a, b = divmod(i,n)
    print(a,b)
    
    #flush is to the right
    position_x = str(float(syringe_position_x)-(sample_distance*int(b))-flush_d)
    position_y = str(float(syringe_position_y)-(sample_distance*int(a)))
    print(f'droplet center:{position_x,position_y}')
   
    #Here p (ratio) should change with optimization
    V_0 = p[i] * V_d
    V_1 = (1-p[i]) * V_d 

    Q_0 = 60 * (V_0/t_d)
    Q_1 = 60 * (V_1/t_d)
    
    print(f'droplet volume:{V_0,V_1}')
    
    #move droplet i location to syringe and start experiment
    move_to(position_x,position_y)
    single_pump(sy_dia, res_vol, 0, Q_0, V_0)
    time.sleep(0.2)
    single_pump(sy_dia, res_vol, 1, Q_1, V_1)
    time.sleep(0.2)
    
    #allow droplet evaporation 
    time.sleep(e_t)
    
    time.sleep(10)
    
    #move to camera and image
    position_x = str(float(camera_position_x)-(sample_distance*int(b))-flush_d)
    position_y = str(float(camera_position_y)+(sample_distance*int(a)))
    move_to(position_x,position_y)
    time.sleep(0.2)
    time.sleep(10)
    #capture_and_save_image(i)
    time.sleep(0.2)
    
    #image processing
    
    #optimization 
    print('end of loop')

In [ ]:
position_x = str(float(syringe_position_x)-(0)-flush_d)
position_y = str(float(syringe_position_y)+(1))
    
    #move droplet i location to syringe and start experiment
move_to(position_x,position_y)